In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv("data/Train.csv")
X = df.drop(columns=["total_cost"])
y = df["total_cost"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=41, stratify=X[["main_activity"]]
)

Aus der Analyse Phase hatten wie NULL hier:
ID 0
country 0
age_group 0
travel_with 1114
total_female 3
total_male 5
purpose 0
main_activity 0
info_source 0
tour_arrangement 0
package_transport_int 0
package_accomodation 0
package_food 0
package_transport_tz 0
package_sightseeing 0
package_guided_tour 0
package_insurance 0
night_mainland 0
night_zanzibar 0
payment_mode 0
first_trip_tz 0
most_impressing 313


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

X_train["travel_with"].unique()

<StringArray>
['Friends/Relatives', nan, 'Alone', 'Spouse', 'Spouse and Children',
 'Children']
Length: 6, dtype: str

In [ ]:
travel_with_imputer = SimpleImputer(
    missing_values=np.nan, fill_value="Unknown", strategy="constant"
)

travel_with_imputer.fit(X_train[["travel_with"]])

,"missing_values missing_values: int, float, str, np.nan, None or pandas.NA, default=np.nanThe placeholder for the missing values. All occurrences of`missing_values` will be imputed. For pandas' dataframes withnullable integer dtypes with missing values, `missing_values`can be set to either `np.nan` or `pd.NA`.",nan
,"strategy strategy: str or Callable, default='mean'The imputation strategy.- If ""mean"", then replace missing values using the mean along each column. Can only be used with numeric data.- If ""median"", then replace missing values using the median along each column. Can only be used with numeric data.- If ""most_frequent"", then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.- If ""constant"", then replace missing values with fill_value. Can be used with strings or numeric data.- If an instance of Callable, then replace missing values using the scalar statistic returned by running the callable over a dense 1d array containing non-missing values of each column... versionadded:: 0.20 strategy=""constant"" for fixed value imputation... versionadded:: 1.5 strategy=callable for custom value imputation.",'constant'
,"fill_value fill_value: str or numerical value, default=NoneWhen strategy == ""constant"", `fill_value` is used to replace alloccurrences of missing_values. For string or object data types,`fill_value` must be a string.If `None`, `fill_value` will be 0 when imputing numericaldata and ""missing_value"" for strings or object data types.",'Unknown'
,"copy copy: bool, default=TrueIf True, a copy of X will be created. If False, imputation willbe done in-place whenever possible. Note that, in the following cases,a new copy will always be made, even if `copy=False`:- If `X` is not an array of floating values;- If `X` is encoded as a CSR matrix;- If `add_indicator=True`.",True
,"add_indicator add_indicator: bool, default=FalseIf True, a :class:`MissingIndicator` transform will stack onto outputof the imputer's transform. This allows a predictive estimatorto account for missingness despite imputation. If a feature has nomissing values at fit/train time, the feature won't appear onthe missing indicator even if there are missing values attransform/test time.",False
,"keep_empty_features keep_empty_features: bool, default=FalseIf True, features that consist exclusively of missing values when`fit` is called are returned in results when `transform` is called.The imputed value is always `0` except when `strategy=""constant""`in which case `fill_value` will be used instead... versionadded:: 1.2",False


In [ ]:
travel_with_imputer.statistics_

array(['Unknown'], dtype=object)

In [ ]:
travel_with_train = travel_with_imputer.transform(X_train[["travel_with"]])

In [ ]:
total_sex_imputer1 = SimpleImputer(
    missing_values=np.nan, fill_value=0.0, strategy="constant"
)
total_sex_imputer2 = SimpleImputer(
    missing_values=np.nan, fill_value=0.0, strategy="constant"
)

In [ ]:
total_female_train = total_sex_imputer1.fit_transform(X_train[["total_female"]])
total_male_train = total_sex_imputer2.fit_transform(X_train[["total_male"]])

In [ ]:
X_train["most_impressing"].unique()

<StringArray>
[                     'Friendly People',
                          'No comments',
 'Wonderful Country, Landscape, Nature',
                            ' Wildlife',
                 'Excellent Experience',
                                    nan,
                         'Good service',
         'Satisfies and Hope Come Back']
Length: 8, dtype: str

In [ ]:
most_impressing_imputer = SimpleImputer(
    missing_values=np.nan, fill_value="No comments", strategy="constant"
)

In [ ]:
most_impressing_train = most_impressing_imputer.fit_transform(
    X_train[["most_impressing"]]
)

In [ ]:
one_hot_encoder1 = OneHotEncoder(
    handle_unknown="ignore", sparse_output=False, drop="first"
)
one_hot_encoder2 = OneHotEncoder(
    handle_unknown="ignore", sparse_output=False, drop="first"
)

In [ ]:
travel_with_encoded_train = one_hot_encoder1.fit_transform(travel_with_train)

In [ ]:
most_impressing_encoded_train = one_hot_encoder2.fit_transform(most_impressing_train)

In [ ]:
features_categories = [
    "country",
    "purpose",
    "main_activity",
    "info_source",
    "tour_arrangement",
    "package_transport_int",
    "package_accomodation",
    "package_food",
    "package_transport_tz",
    "package_sightseeing",
    "package_guided_tour",
    "package_insurance",
    "payment_mode",
    "first_trip_tz",
]
encoded_fetures = []
one_hot_encoder = dict()
for feature in features_categories:
    one_hot_encoder[feature] = OneHotEncoder(
        handle_unknown="ignore", sparse_output=False, drop="first"
    )
    encoded_feature = one_hot_encoder[feature].fit_transform(X_train[[feature]])
    encoded_fetures.append(encoded_feature)

encoded_fetures = pd.DataFrame(np.concatenate(encoded_fetures, axis=1))

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

ordinal_encoded_train = ordinal_encoder.fit_transform(X_train[["age_group"]])

In [ ]:
X_train_encoded = pd.DataFrame(
    np.concatenate(
        [
            encoded_fetures.values,
            ordinal_encoded_train,
            travel_with_encoded_train,
            total_male_train,
            total_female_train,
            most_impressing_encoded_train,
        ],
        axis=1,
    )
)

In [ ]:
X_train_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,135,136,137,138,139,140,141,142,143,144
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


Finales Modell trainieren und bewerten


In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

# transform test data
travel_with_test = travel_with_imputer.transform(X_test[["travel_with"]])
total_female_test = total_sex_imputer1.transform(X_test[["total_female"]])
total_male_test = total_sex_imputer2.transform(X_test[["total_male"]])
travel_with_encoded_test = one_hot_encoder1.transform(travel_with_test)
most_impressing_test = one_hot_encoder2.transform(X_test[["most_impressing"]])
encoded_fetures_test = []
for feature in features_categories:
    encoded_feature_test = one_hot_encoder[feature].transform(X_test[[feature]])
    encoded_fetures_test.append(encoded_feature_test)
encoded_fetures_test = pd.DataFrame(np.concatenate(encoded_fetures_test, axis=1))
ordinal_encoded_test = ordinal_encoder.transform(X_test[["age_group"]])
X_test_encoded = pd.DataFrame(
    np.concatenate(
        [
            encoded_fetures_test.values,
            ordinal_encoded_test,
            travel_with_encoded_test,
            total_male_test,
            total_female_test,
            most_impressing_test,
        ],
        axis=1,
    )
)


sgd_regressor = Ridge(alpha=100, random_state=42)
sgd_regressor.fit(X_train_encoded, y_train)

from sklearn.metrics import mean_squared_error

y_test_pred = sgd_regressor.predict(X_test_encoded)
mse_train = mean_squared_error(y_test, y_test_pred)
print(f"Mean Squared Error on Training Set: {mse_train}")

Mean Squared Error on Training Set: 120824535439716.66


c:\Users\benzenhoefer\KI_NF\Tanzania-Tourism-Prediction\.venv\Lib\site-packages\sklearn\utils\validation.py:2684: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(
c:\Users\benzenhoefer\KI_NF\Tanzania-Tourism-Prediction\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
c:\Users\benzenhoefer\KI_NF\Tanzania-Tourism-Prediction\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.ensemble import ExtraTreesRegressor

X_train_encoded_2 = pd.get_dummies(X_train.drop(["ID", "country"], axis=1)).fillna(0)
X_test_encoded_2 = pd.get_dummies(X_test.drop(["ID", "country"], axis=1)).fillna(0)


y_train_scaled = np.log1p(y_train)
y_test_scaled = np.log1p(y_test)

rfr = ExtraTreesRegressor()
rfr.fit(X_train_encoded_2, y_train_scaled)

y_test_pred_rfr = rfr.predict(X_test_encoded_2)
mse_test_rfr = mean_squared_error(y_test, y_test_scaled)
print(f"Mean Squared Error on Test Set (Random Forest Regressor): {mse_test_rfr}")

Mean Squared Error on Test Set (Random Forest Regressor): 228034347499769.75


In [58]:
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OrdinalEncoder


X = df.drop(["ID", "total_cost"], axis=1)
y = df["total_cost"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

imputter = SimpleImputer(missing_values=np.nan, fill_value=0.0, strategy="constant")
one_hot_encoder = OneHotEncoder(
    handle_unknown="ignore", sparse_output=False, drop="first"
)
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

transformers = [
    (
        "imputter",
        imputter,
        ["total_female", "total_male", "night_mainland", "night_zanzibar"],
    ),
    (
        "ohe",
        one_hot_encoder,
        [
            "country",
            "purpose",
            "travel_with",
            "main_activity",
            "info_source",
            "tour_arrangement",
            "package_transport_int",
            "package_accomodation",
            "package_food",
            "package_transport_tz",
            "package_sightseeing",
            "package_guided_tour",
            "package_insurance",
            "payment_mode",
            "first_trip_tz",
            "most_impressing"
        ],
    ),
    ("ord", ordinal_encoder, ["age_group"]),
]

column_transformer = ColumnTransformer(
    transformers=transformers, remainder="passthrough"
).set_output(transform="pandas")

scaler = StandardScaler()

column_transformer_2 = ColumnTransformer(
    transformers=[("scaling", scaler, "imputter__" + X_train.select_dtypes(include='number').columns)], remainder="passthrough"
).set_output(transform="pandas")

pipeline = Pipeline(
    steps=[
        ("preprocessor", column_transformer),
        ("scaler", column_transformer_2),
        
    ]
)

pipeline.fit(X_train)


column_transformer_y = StandardScaler()

X_train_fe = pipeline.transform(X_train)
X_test_fe = pipeline.transform(X_test)

column_transformer_y.fit(y_train.values.reshape(-1, 1))

y_train_fe = column_transformer_y.transform(y_train.values.reshape(-1, 1))
y_test_fe = column_transformer_y.transform(y_test.values.reshape(-1, 1))


c:\Users\benzenhoefer\KI_NF\Tanzania-Tourism-Prediction\.venv\Lib\site-packages\sklearn\preprocessing\_encoders.py:261: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


In [ ]:

from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.0001, tol=1e-10)

lasso.fit(X_train_fe, y_train)

y_test_pred_lasso = lasso.predict(X_test_fe)
mse_test_lasso = mean_squared_error(y_test, y_test_pred_lasso)
mae_test_lasso = mean_absolute_error(y_test, y_test_pred_lasso)
print(f"Mean Squared Error on Test Set (Lasso): {mse_test_lasso}")
print(f"Mean Absolute Error on Test Set (Lasso): {mae_test_lasso}")

In [63]:
from sklearn.ensemble import GradientBoostingRegressor

gnr = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.01, n_estimators=1000, max_depth=3)

gnr.fit(X_train_fe, y_train)

y_test_pred_gnr = gnr.predict(X_test_fe)
mse_test_gnr = mean_squared_error(y_test, y_test_pred_gnr)
mae_test_gnr = mean_absolute_error(y_test, y_test_pred_gnr)
print(f"Mean Squared Error on Test Set (GradientBoostingRegressor): {mse_test_gnr}")
print(f"Mean Absolute Error on Test Set (GradientBoostingRegressor): {mae_test_gnr}")

Mean Squared Error on Test Set (GradientBoostingRegressor): 87694323990587.67
Mean Absolute Error on Test Set (GradientBoostingRegressor): 4515755.599460565


Mean Squared Error on Test Set (GradientBoostingRegressor): 87642526680211.62     
Mean Absolute Error on Test Set (GradientBoostingRegressor): 4532948.634085677

Mean Squared Error on Test Set (GradientBoostingRegressor): 86756641033741.2     
Mean Absolute Error on Test Set (GradientBoostingRegressor): 4517485.804137394